#### 라이브러리 및 환경변수 세팅

In [1]:
import numpy as np
import pandas as pd
import os
from google.analytics.data_v1beta import BetaAnalyticsDataClient 
from google.analytics.data_v1beta.types import (
    RunReportRequest, 
    Dimension, 
    Metric,
    OrderBy, 
    DateRange, 
)

In [ ]:
# os 모듈을 사용하여 환경 변수를 설정할 수 있습니다.
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'GCP에서_생성한_json_파일.json'
# Google Analytics 4 속성 ID 설정
property_id = 'GA4_속성_ID'
# Google Analytics Data API 클라이언트 초기화. 이 클라이언트를 통해 API에 요청을 보내고 데이터를 받을 수 있습니다.
client = BetaAnalyticsDataClient()

### 내림차순 정렬하기
[GA4 API 공식문서](https://developers.google.com/analytics/devguides/reporting/data/v1/rest/v1beta/OrderBy?hl=ko)에서 OrderBy 사용법을 볼 수 있다.

문서에 의하면 OrderBy 클래스에서 사용할 수 있는 필드는<br>
desc, metric, dimention, pivot으로 총 4가지로 설명되어있다. 

앞서 예시로 살펴보았던 일별 DAU를 내림차순으로 설정해보자.

In [22]:
request = RunReportRequest(
        property='properties/'+property_id,
        dimensions=[Dimension(name="date")],
        metrics=[Metric(name="active1DayUsers")],
        order_bys = [OrderBy(dimension = {'dimension_name': 'date'}, desc=True)],  # desc 필드를 True로 설정
        date_ranges=[DateRange(start_date="2024-01-01", end_date="2024-02-01")],
    )

In [24]:
def dataframe(request):
    response = client.run_report(request)
    data = [] 
    for row in response.rows:
        row_data = {dim.name: row.dimension_values[i].value for i, dim in enumerate(response.dimension_headers)}
        row_data.update({metric.name: float(row.metric_values[i].value) for i, metric in enumerate(response.metric_headers)})
        data.append(row_data)
    df = pd.DataFrame(data)
    print(df.head(1))

dataframe(request)

       date  active1DayUsers
0  20240201          81312.0


### 문자열 측정기준 값의 순서 지정하기
dimension의 경우 값이 문자열 타입인 경우가 많기 때문에 이 경우엔 값의 순서를 지정할 수 있다.

열거형                         | 설명
---------------------------- | ---------------------------------------------------------
ORDER_TYPE_UNSPECIFIED       | 지정되지 않았습니다.
ALPHANUMERIC                  | 유니코드 코드 포인트별로 영숫자로 정렬합니다. 예: '2' < 'A' < 'X' < 'b' < 'z'
CASE_INSENSITIVE_ALPHANUMERIC | 대소문자를 구분하지 않는 영숫자로 된 소문자 유니코드 코드 포인트로 정렬합니다. 예: '2' < 'A' < 'b' < 'X' < 'z'
NUMERIC                       | 측정기준 값은 정렬 전에 숫자로 변환됩니다. 예를 들어, 숫자 정렬에서는 '25' < '100'입니다. <br>ALPHANUMERIC 정렬에서는 '100' < '25'. 숫자가 아닌 측정기준 값은 모두 모든 숫자 값보다 낮은 순서 값을 갖습니다.

In [37]:
request = RunReportRequest(
        property='properties/'+property_id,
        dimensions=[Dimension(name="country")],
        metrics=[Metric(name="active1DayUsers")],
        order_bys = [OrderBy(dimension = {'dimension_name': 'country', 'order_type': 'NUMERIC'}, desc=True)],  # desc 필드를 True로 설정
        date_ranges=[DateRange(start_date="2024-01-01", end_date="2024-02-01")],
    )
dataframe(request)

       country  active1DayUsers
0  South Korea        1643640.0


### 측정항목 기준 내림차순
이번엔 측정항목 기준으로 내림차순을 해보자.<br>
[MetricOrderBy](https://developers.google.com/analytics/devguides/reporting/data/v1/rest/v1beta/OrderBy?hl=ko#metricorderby) JSON 표현식에 맞게 설정해준다.

In [25]:
request = RunReportRequest(
        property='properties/'+property_id,
        dimensions=[Dimension(name="date")],
        metrics=[Metric(name="active1DayUsers")],
        order_bys = [OrderBy(metric = {'metric_name': 'active1DayUsers'}, desc=True)],  # metric 필드를 json표현식에 맞게 설정
        date_ranges=[DateRange(start_date="2024-01-01", end_date="2024-02-01")],
    )

dataframe(request)

       date  active1DayUsers
0  20240108          92634.0
